### DNR Data Filtration

The block of code below is used to clean and filter the raw DNR reports for water quality data of various Wisconsin lakes. The script loops through all the raw data files, extracts the lake name, the county the lake is in, the WBIC code (used later to identify the lake), and the "lake index" which is just the loop location. The script removes all the lines in the original CSV except those that contain the secchi depth, chlorophyll-a, and total phosphorus measurements for the date. These values are then saved to a CSV and placed in a folder identified by the lake index. 

In [20]:
import os
import re
import glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

# Make list of all reports
files = glob.glob("original_lake_reports/repor*.csv")

# Define an empty dataframe that will store general information for each lake
lake_df = pd.DataFrame(columns=["lakename","county", "WBIC", "lake_index"])

# Loop through all files
for i in tqdm(range(len(files))):
    # set index of lake
    index = i
    
    # Read in report csv
    report_df = pd.read_csv(f"{files[i]}", skiprows=8,error_bad_lines=False, index_col=False,usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])
    # Rename Columns so they're easier to work with
    report_df = report_df.rename(columns={" Start Date":"date","Secchi (Meters)":"secchi_m", "Chlorophyll(ug/l)":"chla","Total Phosphorus(ug/l)":"TP"})
    
    # Remove any columns that don't have either a TP or chla value
    report_df = report_df.loc[(report_df.TP.notna()  | report_df.chla.notna() | report_df.secchi_m.notna())][["date", "secchi_m", "chla", "TP", "Perception"]]
    report_df = report_df.reset_index(drop=True)
    # Remove the bottom of the dataframe that is now measuring DO and reset index
    cut_index = report_df.loc[report_df.date == "Depth"].index[0]
    report_df = report_df[:cut_index]
    
    
    # Make perception column equal to integer value
    perception_series = report_df.Perception.notna()
    for j in range(len(report_df)):
        if perception_series[j] == True:
            report_df.Perception.iloc[j] = report_df.Perception.iloc[j][0]
            if report_df.Perception.iloc[j] == " ":
                report_df.Perception.iloc[j] = np.nan
            else:
                report_df.Perception.iloc[j] = int(report_df.Perception.iloc[j])   
    
    # Make date column a manageable time series
    report_df["date"] = pd.to_datetime(report_df["date"])
    
    # define a dataframe which will contain only the header of the lake reports; this is how we extract 
    # the lake name, county, and WBIC
    df_name = pd.read_csv(f"{files[i]}", nrows=1,skiprows= 3,error_bad_lines=False)
    df_name = df_name.rename(columns={"Lake Name":"lakename", "County Name":"county", "Waterbody ID(WBIC)":"WBIC"})
    df_name = df_name[["lakename","county","WBIC"]]
    
    # Add the new lake to the lake_df dataframe
    lake_df = lake_df.append({"lakename":df_name.lakename.iloc[0], "county":df_name.county.iloc[0], "WBIC":df_name.WBIC.iloc[0], "lake_index":i},ignore_index=True)
    # Make a new folder to save the lake data to
    os.mkdir(f"C:/Users/dlcole3/Documents/UW_Madison/Research/Models/Pmodel/lake_reports_full/Lakes/lake{i}")
    # Save the lake data
    report_df.to_csv(f"lake_reports_full/Lakes/lake{i}/DNR_data.csv",index=False)
    
# Save the dataframe that contains the lake index, lake name, county, and WBIC code
lake_df.to_csv("lake_index_WBIC.csv", index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 871/871 [00:59<00:00, 14.63it/s]


### Adding the COMID to the lake list

The DNR identifies their lake data by the WBIC code. Unfortunately, this code is not used the NHDPlusV2 dataset which we use for identifying waterbodies. To overcome this, we needed a way to match COMID to WBIC code. We do this by using the Hydro Waterbodies dataset, which has shapefiles identified by WBIC codes. In this next block of code, we match the shapefiles of the Hydro Waterbodies set to the shapefiles of the NHDPlusV2 set. We first test whether the centroid of the Hydro Waterbodies are within the NHDPlusV2 waterbodies. This is a fast test, but because of the shapes of some lakes, the centroid lies outside of the waterbody. Therefore, after testing the centroid, we test whether the Hydro Waterbodies intersect any of the NHDPlusV2 waterbodies, and we make sure that if they do, there is only one waterbody which intersects. 

In [4]:
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Read in NHDPlusV2 waterbodies
WILakes = pd.read_pickle("../graph_construction/WILakes.df")

# Read in the hydro waterbodies shapefiles and set the crs to match the EPSG code of the NHDPlusV2 dataset
hw = gpd.GeoDataFrame.from_file("Hydro_Waterbodies/24k_Hydro_Waterbodies_(Open_Water).shp")
hw = hw.to_crs("EPSG:4326")

# Read in the lake_index CSV
lake_index = pd.read_csv("lake_index_WBIC.csv")

# Take a slice of the dataframe so that we are only testing the lakes which show up in the lake_index
hw_new = hw[hw.WATERBOD_2.isin(lake_index.WBIC.values)]
hw_new = hw_new.reset_index(drop=True)

# Define a set of geometries for the NHDPlusV2 data
polys = WILakes.geometry

# Add column to the lake_index dataframe that will contain the given COMID
lake_index["COMID"] = 0

# Loop through all lakes in the lake_index
for i in tqdm(range(len(lake_index))):
    # Get the WBIC code of the lake index
    WBIC    = lake_index.WBIC.iloc[i]
    
    # Get the hydro waterbodies slice that contains the lake index WBIC code.
    hw_WBIC = hw_new[hw_new.WATERBOD_2 == WBIC]
    
    # If the WBIC code of the lake_index set has no entry, then we skip that lake and we cannot 
    # find a COMID that matchs
    if len(hw_WBIC) == 0:
        continue
        
    # Otherwise, get the centroid of the Hydro Waterbody that matches the WBIC code of the lake_index set
    if len(hw_WBIC) > 0:
        point   = hw_WBIC.centroid.iloc[0]
    
    # Loop through the NHDPlusV2 set of lakes, and see if the centroid is inside an NHDPlusV2 lake
    for j in range(len(WILakes)):
        
        # If the centroid is within the NHDPlusV2 lake, set the COMID to match that lake
        if point.within(polys.iloc[j]):
            lake_index.COMID.iloc[i] = WILakes.COMID.iloc[j]
            break

    # If the COMID of the lake_index set has been reset, move on to the next iteration
    if lake_index.COMID.iloc[i] != 0:
        pass

    # Otherwise, the centroid may lie outside of the NHDPlusV2 lake. Instead, test if it intersects. It is possible that
    # multiple NHDPlusV2 lakes intersect the same Hydro Waterbody. If this is the case, we will not set a COMID because we 
    # cannot tell which lake it corresponds to. 
    else:
        comid_val = []
        counter = 0
        for j in range(len(WILakes)):
            if hw_WBIC.geometry.iloc[0].intersects(polys.iloc[j]):
                comid_val = np.append(comid_val, WILakes.COMID.iloc[j])
                counter += 1

        if counter == 1:
            lake_index.COMID.iloc[i] = comid_val[0]

# Save the new lake_index dataframe that now contains COMIDs
lake_index.to_csv("lake_index_WBIC_COMID.csv",index=False)


100%|████████████████████████████████████████████████████████████████████████████████| 871/871 [01:38<00:00,  8.80it/s]
